# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cleancity_data_df=pd.read_csv('../output_data/CityData.csv')
cleancity_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,upernavik,72.7868,-56.1549,49.10,75,98,8.81,GL,1627440266
1,iquique,-20.2208,-70.1431,59.04,74,22,4.03,CL,1627440864
2,avarua,-21.2078,-159.7750,77.05,65,21,4.61,CK,1627440269
3,port alfred,-33.5906,26.8910,60.28,83,11,14.32,ZA,1627440285
4,rikitea,-23.1203,-134.9692,74.21,79,100,15.30,PF,1627440270


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps & Create Location Variables 
gmaps.configure(api_key=g_key)

locations = cleancity_data_df[["Lat", "Lng"]]
humidity = cleancity_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = cleancity_data_df.loc[(cleancity_data_df["Max Temp"] < 80) & (cleancity_data_df["Max Temp"] > 70) \
                                    & (cleancity_data_df["Wind Speed"] < 10) \
                                    & (cleancity_data_df["Cloudiness"] == 0)].dropna()
narrow_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,filadelfia,39.9523,-75.1638,79.09,67,0,4.97,US,1627440761
72,hovd,48.0056,91.6419,74.91,30,0,3.65,MN,1627440885
88,miracema do tocantins,-9.5618,-48.3967,74.32,41,0,3.31,BR,1627440890
112,awjilah,29.1081,21.2869,78.89,47,0,5.68,LY,1627440898
141,gat,31.6100,34.7642,77.68,100,0,1.72,IL,1627440617
168,jalu,29.0331,21.5482,79.41,44,0,5.44,LY,1627440915
244,tukrah,32.5341,20.5791,75.16,77,0,9.44,LY,1627440938
247,rosolini,36.8242,14.9478,74.05,41,0,3.71,IT,1627440939
262,minna,9.6139,6.5569,73.11,94,0,0.00,NG,1627440818
390,findlay,41.0442,-83.6499,75.65,84,0,4.34,US,1627440985


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
41,filadelfia,US,39.9523,-75.1638,
72,hovd,MN,48.0056,91.6419,
88,miracema do tocantins,BR,-9.5618,-48.3967,
112,awjilah,LY,29.1081,21.2869,
141,gat,IL,31.6100,34.7642,
168,jalu,LY,29.0331,21.5482,
244,tukrah,LY,32.5341,20.5791,
247,rosolini,IT,36.8242,14.9478,
262,minna,NG,9.6139,6.5569,
390,findlay,US,41.0442,-83.6499,


In [22]:
MainUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params   = {"radius": 5000,"type": "lodging","key": g_key,}
# Data Manipulation
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(MainUrl, params=params).json()   
    results  = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")

Retrieving Results for Index 41: filadelfia.
Missing field/result... skipping.
------------
Retrieving Results for Index 72: hovd.
Missing field/result... skipping.
------------
Retrieving Results for Index 88: miracema do tocantins.
Missing field/result... skipping.
------------
Retrieving Results for Index 112: awjilah.
Missing field/result... skipping.
------------
Retrieving Results for Index 141: gat.
Closest hotel is OlusHome.
------------
Retrieving Results for Index 168: jalu.
Missing field/result... skipping.
------------
Retrieving Results for Index 244: tukrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 247: rosolini.
Closest hotel is B&B Home Sweet Home Rosolini.
------------
Retrieving Results for Index 262: minna.
Missing field/result... skipping.
------------
Retrieving Results for Index 390: findlay.
Missing field/result... skipping.
------------
Retrieving Results for Index 410: hoquiam.
Missing field/result... skipping.
------------
Re

In [23]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
41,filadelfia,US,39.9523,-75.1638,
72,hovd,MN,48.0056,91.6419,
88,miracema do tocantins,BR,-9.5618,-48.3967,Miami Apart Hotel
112,awjilah,LY,29.1081,21.2869,
141,gat,IL,31.6100,34.7642,OlusHome
168,jalu,LY,29.0331,21.5482,
244,tukrah,LY,32.5341,20.5791,
247,rosolini,IT,36.8242,14.9478,B&B Home Sweet Home Rosolini
262,minna,NG,9.6139,6.5569,
390,findlay,US,41.0442,-83.6499,


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))